# Google Colaboratory の準備

本ファイルは、`notebook/1-3_train.ipynb`をGoogle Colaboratoryで実行するためのノートブックです。  
あらかじめ、`notebook/colab/1-0_prepare_colab.ipynb`の手順を実行し、Googleドライブにリファレンス環境（ソフトウェア）のリポジトリとデータセットの準備が完了していることを確認してください。

## GPUの指定

Google Colaboratoryのメニューバーの「ランタイム」から「ランタイムのタイプを変更」を選択し、
ハードウェア アクセラレータを**GPU**に指定した上で、本ファイルを実行してください。

## ライブラリのインストール

In [ ]:
!pip install -q torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q nuscenes-devkit

!pip install -q fire tensorboardX protobuf opencv-python
!pip install -q numba scikit-image scipy pillow

     |███████████████████████         | 834.1 MB 1.3 MB/s eta 0:04:05tcmalloc: large alloc 1147494400 bytes == 0x390b0000 @  0x7fa267c84615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████████████▏  | 1055.7 MB 1.2 MB/s eta 0:01:26tcmalloc: large alloc 1434370048 bytes == 0x7d706000 @  0x7fa267c84615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 1156.7 MB 1.3 MB/s eta 0:00:01tcmalloc: large alloc 1445945344 bytes == 0xd2ef2000 @  0x7fa267c84615 0x592b76 0x4df71e 0x59afff 0x515655 0x

In [ ]:
# spconv v1.2.1 のインストール

!git clone --depth 1 --recursive  -b v1.2.1 https://github.com/traveller59/spconv.git
%cd spconv
!SPCONV_FORCE_BUILD_CUDA=1 python setup.py bdist_wheel
!pip install ./dist/spconv*

Cloning into 'spconv'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 118 (delta 16), reused 32 (delta 1), pack-reused 0
Receiving objects: 100% (118/118), 234.67 KiB | 6.17 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Submodule 'third_party/cutlass' (https://github.com/NVIDIA/cutlass) registered for path 'third_party/cutlass'
Submodule 'third_party/mp11' (https://github.com/boostorg/mp11) registered for path 'third_party/mp11'
Submodule 'third_party/pybind11' (https://github.com/pybind/pybind11.git) registered for path 'third_party/pybind11'
Cloning into '/content/spconv/third_party/cutlass'...
remote: Enumerating objects: 16512, done.        
remote: Counting objects: 100% (567/567), done.        
remote: Compressing objects: 100% (297/297), done.        
remote: Total 16512 (delta 257), reused 529 (delta 249), pack-reused 15945        
Receiving objects: 100% (16512/1651

## Googleドライブをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# WORK_DIRのパスは自身の作業ディレクトリに変更してください。

WORK_DIR = "/content/drive/MyDrive/Colab Notebooks/work/"
%cd $WORK_DIR

# パスを合わせるため、リポジトリのnotebookディレクトリに移動
%cd ./6th-ai-reference2/notebook

/content/drive/MyDrive/Colab Notebooks/work
/content/drive/MyDrive/Colab Notebooks/work/6th-ai-reference2/notebook


以下より、`notebook/1-3_train.ipynb`と同じ内容です。

# 概要

本ファイルは、3D物体検出モデルの学習を行います。    

> 本ファイルで実行するスクリプトはsecond.pytorchをベースとしています。

## Output

本ファイルを実行することで、`second.pytorch/checkpoints/{model_dir}` ディレクトリに学習結果が保存されます。  
> `{model_dir}`は、ノートブック内で設定する保存ディレクトリ名です。任意のディレクトリ名に変更して実行できます。  
> デフォルトのディレクトリ名は、`model`として設定されています。  
> ただし、既に存在するディレクトリ名を指定するとエラーとなります。


```
└ second.pytorch
     └─ checkpoints
          └ {model_dir}                   <- 本ファイルにて生成されます
                ├── summary             <- Tensorboard用ログ
                ├── checkpoints.json
                ├── log.txt
                ├── {model-step}.tckpt  <- 学習済みモデル
```

# 準備

あらかじめ、 `data`ディレクトリに、`3d_labels_painted`ディレクトリと、各メタ情報(pkl)ファイルがあることを確認してください。

```
└ data
    ├ 3d_labels
    └ 3d_labels_painted
        ├ maps
        ├ samples
            ├ CAM_FRONT
            └ LIDAR_TOP
        ├ v1.0-trainval
        ├ gt_database        <- 1-2_create_dataset_metadata_colab.ipynb にて生成
        ├ infos_train.pkl    <- 1-2_create_dataset_metadata_colab.ipynb にて生成
        ├ infos_val.pkl      <- 1-2_create_dataset_metadata_colab.ipynb にて生成
        └ kitti_dbinfos_train.pkl  <- 1-2_create_dataset_metadata_colab.ipynb にて生成
```

# 学習

## （オプション）Tensorboardの起動

学習結果はTensorboardで確認することができます。  
Tenosorboardを起動しなくても、学習は実行できます。

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

%tensorboard --logdir "../second.pytorch/checkpoints/" --bind_all

## 学習の実行


データセット(3d_labels)の学習データに対して学習を実施します。  

> 学習データと検証データはnuScenesデータセットで定義されています。  
> 検証データはscene-0094とscene-0109 の2シーンで、
> 学習データはそれ以外の35シーンです。

In [ ]:
# パスの設定

config_path = "../second.pytorch/second/configs/nuscenes/pointpainting.config"

model_dir = "../second.pytorch/checkpoints/model"

In [ ]:
# 学習スクリプトの実行

!python ../second.pytorch/second/pytorch/train.py train \
        --config_path=$config_path --model_dir=$model_dir

以上